<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/getHAFSASlices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program subsets and saves portions of output from the MOM6 model along the track of a given TC.

#Set up environment

In [1]:
!pip install cfgrib
!pip install cartopy
!pip install tropycal

In [2]:
!sudo apt install aria2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
from tropycal import tracks, rain
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime as dt
import cfgrib
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#User parameters

In [5]:
name = "Milton"
tcNum = "14"
filepath = f"/content/drive/MyDrive/ColabNotebooks/{name}"
trackType = ""

centralTime = dt(year=2024, month=10, day=7, hour=6) #Time when Milton began its most rapid intensification
daysBefore = 1 #Days before the focal point
daysAfter = 1 #Days after focal point

fHourStep = 6 #Normally 3 for HAFS-A
forecastLength = 36 #Normally 126 for HAFS-A. Changeable for testing.
runStep = 12 #Normally 6 for HAFS-A

figureSuffix = "_RI"

In [6]:
atmTop = 700 # in hPa
oceBottom = 530 # In m below surface. This was chosen to include a layer in the files that's around 529 m

#Determine Area and Temporal Extent

Get best track data and find bounds of TC track

In [7]:
bt = pd.read_csv(filepath + "/hurdat2_" + name + trackType + ".csv")
latBounds = [min(bt.lat), max(bt.lat)]
lonBounds = [min(bt.lon), max(bt.lon)]

In [8]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d%H"

start = centralTime - pd.Timedelta(days=daysBefore)
end = centralTime + pd.Timedelta(days=daysAfter)

Find times needed

In [9]:
fcastTimes = {} #Key: initiation, item: valid time list

initTime = start
while initTime <= end:
  validTime = initTime
  fcastTimes[initTime] = []
  fhour = 0

  while validTime <= end and fhour <= forecastLength:
    fcastTimes[initTime].append(validTime)
    validTime += pd.Timedelta(hours=fHourStep)
    fhour += fHourStep

  initTime += pd.Timedelta(hours=runStep)

#Retrieve Data

In [10]:
bucket = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/"
#atmRunSlices = [] #Holds xarray datasets for each model run
oceRunSlices = []
atmFiles = []
oceFiles = []

In [33]:
for init, validList in fcastTimes.items():

  initDate, initHour = init.strftime("%Y%m%d_%H").split("_")
  atmRun = False

  for valid in validList:

    fhour = str(int((valid-init).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour

    atmURL = bucket + initDate + "/" + initHour + "/" + tcNum + "l." + initDate + initHour + ".hfsa.storm.atm.f" + fhour + ".grb2"
    atmFile = "atm_" + initDate + "_" + initHour + "_f" + fhour + ".grb2"
    atmFiles.append(atmFile)

    !aria2c -x 16 -s 16 --allow-overwrite=true -o {atmFile} {atmURL}

    atmData = xr.open_dataset(atmFile, engine="cfgrib", decode_timedelta=True, filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'isobaricInhPa'})

    pressureSlice = slice(max(atmData.isobaricInhPa.data), atmTop)
    #Longitude in atm files are in degrees east, but are -180 - 180 in oce files. point has them from -180 - 180
    atmSlice = atmData.sel(latitude=slice(latBounds[0], latBounds[1]), longitude=slice(360+lonBounds[0], 360+lonBounds[1]), isobaricInhPa=pressureSlice)

    if atmRun == False: atmRun = atmSlice #DEBUG
    else: atmRun = xr.concat((atmRun, atmSlice), dim="valid_time")

  atmRun.to_netcdf("hafsa_" + initDate + initHour + ".nc")

  #for file in atmFiles:
  #  !rm {file}
  #atmFiles = []

  break


06/16 23:04:54 [NOTICE] Downloading 1 item(s)

06/16 23:04:57 [NOTICE] Download complete: /content/atm_20241006_06_f000.grb2

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
144e99|OK  |   121MiB/s|/content/atm_20241006_06_f000.grb2

Status Legend:
(OK):download completed.


ERROR:cfgrib.dataset:skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 700., 500., 300., 200., 100.]))



06/16 23:05:00 [NOTICE] Downloading 1 item(s)

06/16 23:05:02 [NOTICE] Download complete: /content/atm_20241006_06_f006.grb2

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
15dbc8|OK  |   137MiB/s|/content/atm_20241006_06_f006.grb2

Status Legend:
(OK):download completed.


ERROR:cfgrib.dataset:skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 700., 500., 300., 200., 100.]))



06/16 23:05:10 [NOTICE] Downloading 1 item(s)

06/16 23:05:12 [NOTICE] Download complete: /content/atm_20241006_06_f012.grb2

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
c86c58|OK  |   130MiB/s|/content/atm_20241006_06_f012.grb2

Status Legend:
(OK):download completed.


...

ERROR:cfgrib.dataset:skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 700., 500., 300., 200., 100.]))



06/16 23:05:43 [NOTICE] Downloading 1 item(s)

06/16 23:05:45 [NOTICE] Download complete: /content/atm_20241006_06_f030.grb2

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
18e72f|OK  |   121MiB/s|/content/atm_20241006_06_f030.grb2

Status Legend:
(OK):download completed.


ERROR:cfgrib.dataset:skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 700., 500., 300., 200., 100.]))



06/16 23:05:55 [NOTICE] Downloading 1 item(s)

06/16 23:05:58 [NOTICE] Download complete: /content/atm_20241006_06_f036.grb2

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
368e2a|OK  |   126MiB/s|/content/atm_20241006_06_f036.grb2

Status Legend:
(OK):download completed.


ERROR:cfgrib.dataset:skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 700., 500., 300., 200., 100.]))


In [41]:
atmRun.step.data

array(129600000000000, dtype='timedelta64[ns]')

In [40]:
atmSlice.step.data

array(129600000000000, dtype='timedelta64[ns]')

In [28]:
atmRun = xr.open_dataset("hafsa_" + initDate + initHour + ".nc", decode_timedelta=False)

In [ ]:
for init, validList in fcastTimes.items():

  initDate, initHour = init.strftime("%Y%m%d_%H").split("_")
  oceRun = False

  for valid in validList:

    fhour = str(int((valid-init).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour

    oceURL = bucket + initDate + "/" + initHour + "/" + tcNum + "l." + initDate + initHour + ".hfsa.mom6.f" + fhour + ".nc"
    oceFile = "oce_" + initDate + "_" + initHour + "_f" + fhour + ".nc"

    #if oceFile != 'oce_20241008_00_f000.nc': #This file is missing
    !aria2c -x 16 -s 16 --allow-overwrite=true -o {oceFile} {oceURL}
    oceData = xr.open_dataset(oceFile, decode_times=False)

    depthSlice = slice(min(oceData.z_l.data), oceBottom)
    oceSlice = oceData.sel(z_l=depthSlice, z_i=depthSlice,\
                           xh=slice(lonBounds[0],lonBounds[1]), yh=slice(latBounds[0],latBounds[1]),\
                           xq=slice(lonBounds[0],lonBounds[1]), yq=slice(latBounds[0],latBounds[1]))

    if oceRun == False: oceRun = oceSlice
    else: oceRun = xr.concat((oceRun, oceSlice), dim="valid")

  oceRun.to_netcdf("mom6_" + initDate + initHour + ".nc")

  for file in oceFiles:
    !rm {oceFile}
  oceFiles = []

  !rm {oceFile}